## Solving TicTacToe using MiniMax algorithm

[link](https://cwoebker.com/posts/tic-tac-toe)


### Stats for the game

One could assume that there are 9! different possible game states for Tic Tac Toe. After there would be a winner, players would stop playing so we have to discard those states. Instead of the full 9! = 362,880 states we therefore only have 255,168 possible game states.

131,184 (1st player)
77,904 (2nd player)
46,080 (tie)
This might seem like a lot but for a Computer this is a joke. If we assume symmetries (discarding states already present by rotations or reflections) this changes to just about 26,830 possibilitiess.

There are 138 terminal positions after assuming symmetries.

91 (player 1)
44 (player 2)
3 (tie)

Tic Tac Toe is something called a zero-sum game. A game where the outcome always sums up to zero in the end. Furthermore this is a perfect information game where the game state is completely open to you. You know everything about it, no hidden cards or anything like that. Therefore you can play perfectly since no luck or unknown variables are involved. (Playing perfectly doesn’t mean you will win). For this purpose this means there will always be a tie.

When every single one of us plays Tic Tac Toe it is easy to say what goes on in our minds:

We look at the board and try to predict the future. We try to find the move that will be best for us and worst for our opponent.

Essentially the computer does the exact same thing. It has one important advantage though it can see further into the future (given enough computing power). We could make the computer look far enough so it could see every possible move. This is how the computer can play perfectly and this is how many AI’s related to abstract games (chess, connect four, …) are implemented. If the games are too complex the computer might not be able to compute everything, but we don’t have to worry about that yet.

In [2]:
class Tic(object):
    winning_combos = (
        [0, 1, 2], [3, 4, 5], [6, 7, 8],
        [0, 3, 6], [1, 4, 7], [2, 5, 8],
        [0, 4, 8], [2, 4, 6])

    winners = ('X-win', 'Draw', 'O-win')

    def __init__(self, squares=[]):
        if len(squares) == 0:
            self.squares = [None for i in range(9)]
        else:
            self.squares = squares

    def show(self):
        for element in [self.squares[i:i + 3] for i in range(0, len(self.squares), 3)]:
            print (element)

    def available_moves(self):
        return [k for k, v in enumerate(self.squares) if v is None]

    def available_combos(self, player):
        return self.available_moves() + self.get_squares(player)

    def complete(self):
        if None not in [v for v in self.squares]:
            return True
        if self.winner() != None:
            return True
        return False

    def X_won(self):
        return self.winner() == 'X'

    def O_won(self):
        return self.winner() == 'O'

    def tied(self):
        return self.complete() == True and self.winner() is None

    def winner(self):
        for player in ('X', 'O'):
            positions = self.get_squares(player)
            for combo in self.winning_combos:
                win = True
                for pos in combo:
                    if pos not in positions:
                        win = False
                if win:
                    return player
        return None

    def get_squares(self, player):
        return [k for k, v in enumerate(self.squares) if v == player]

    def make_move(self, position, player):
        self.squares[position] = player

def get_enemy(player):
    if player == 'X':
        return 'O'
    return 'X'

In [6]:
tic = Tic()

tic.show()

tic.make_move(1, 'X')
tic.make_move(8, 'Y')

tic.show()

[None, None, None]
[None, None, None]
[None, None, None]
[None, 'X', None]
[None, None, None]
[None, None, 'Y']


### Game Tree for MiniMax

In [2]:
import copy

In [13]:
def get_successors(current_state, game_tree):
    game_tree_copy = copy.deepcopy(game_tree)
    current_state = game_tree_copy[current_state]
    successor_states = []
    actions = []
    current_state_moves = current_state[0]
    if current_state_moves is None:
        print ("Terminal node")
        return None
    for move, successor in current_state_moves.items():
        if move in possible_moves and move not in actions:
            actions.append(move)
            successor_states.append(successor)
    return successor_states, actions

get_successors('D', game_tree)
            
        

(['K', 'L', 'M'], ['a', 'b', 'c'])

In [15]:
def isTerminal(state, game_tree):
    game_tree_copy = copy.deepcopy(game_tree)
    if game_tree_copy[state][0] is None and game_tree_copy[state][1] is None:
        return True
    return False

isTerminal('K', game_tree)    

True

In [17]:
def get_max_value(state, game_tree):
    game_tree_copy = copy.deepcopy(game_tree)
    # if terminal state, return max value of that state
    if isTerminal(state, game_tree_copy):
        return game_tree_copy[state][2]
    # else return max value from the successors
    succesor_states = get_successors(state, game_tree_copy)[0]
    moves = get_successors(state, game_tree_copy)[1]
    utility_values_of_successors = []
    for state in succesor_states:
        utility_values_of_successors.append(get_min_value(state, game_tree_copy))
    return max(utility_values_of_successors)
    

In [18]:
def get_min_value(state, game_tree):
    game_tree_copy = copy.deepcopy(game_tree)
    # if terminal state, return max value of that state
    if isTerminal(state, game_tree_copy):
        return game_tree_copy[state][2]
    # else return min value from the successors
    succesor_states = get_successors(state, game_tree_copy)[0]
    moves = get_successors(state, game_tree_copy)[1]
    utility_values_of_successors = []
    for state in succesor_states:
        utility_values_of_successors.append(get_max_value(state, game_tree_copy))
    return min(utility_values_of_successors)
    

In [24]:
get_max_value('A', game_tree)

3

In [1]:
game_tree = {
    'A': [{'a': 'B', 'b': 'C', 'c': 'D'}, 1, None],
    'B': [{'a': 'E', 'b': 'F', 'c': 'G'}, -1, None],
    'C': [{'a': 'H', 'b': 'I', 'c': 'J'}, -1, None],
    'D': [{'a': 'K', 'b': 'L', 'c': 'M'}, -1, None],
    'E': [None, None, 3],
    'F': [None, None, 12],
    'G': [None, None, 8],
    'H': [None, None, 2],
    'I': [None, None, 4],
    'J': [None, None, 6],
    'K': [None, None, 14],
    'L': [None, None, 5],
    'M': [None, None, 2]
}

possible_moves = ['a', 'b', 'c']